# Visualisation
First imports:

In [10]:
import pandas as pd

In [11]:
file_path = 'MycroftMind_challenge_dataset.xlsx'

In [12]:
def print_sheet_names(file_path):
    data_file = pd.ExcelFile(file_path)
    names = list(data_file.sheet_names)[1:]
    print(f'Sheet names: {names}')
    for name in names:
        sheet_data = pd.read_excel(file_path, name)
        print(f'\t{name}\'s headers: {sheet_data.columns.to_list()}')

def merge_data(file_path):
    data_file = pd.ExcelFile(file_path)
    sheet_names = list(data_file.sheet_names[1:])
    merged_data = pd.read_excel(file_path, sheet_names[0]) #Imported and exported Energy
    next_data = pd.read_excel(file_path, sheet_names[1]) #Predicted energy
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[2]) #Flexibility
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[3]) #Battery
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID', 'Timestamp'])
    next_data = pd.read_excel(file_path, sheet_names[4]) #Battery params 
    merged_data = pd.merge(merged_data, next_data, on=['DeviceID'], how='left')
    next_data = pd.read_excel(file_path, sheet_names[5]) #Weather
    merged_data = pd.merge(merged_data, next_data, on=['Timestamp'], how='left')
    next_data = pd.read_excel(file_path, sheet_names[6]) #Market prices
    merged_data = pd.merge(merged_data, next_data, on=['Timestamp'], how='left')
    return merged_data

In [13]:
print_sheet_names(file_path)

Sheet names: ['Imported and Exported energy', 'Predicted energy', 'Flexibility', 'Battery', 'Battery parameters', 'Weather', 'Spot Market Prices']
	Imported and Exported energy's headers: ['DeviceID', 'Timestamp', 'Imported_energy (kWh)', 'Exported_energy (kWh)']
	Predicted energy's headers: ['DeviceID', 'Timestamp', 'Predicted_Imported_energy (kWh)', 'Predicted_Exported_energy (kWh)']
	Flexibility's headers: ['DeviceID', 'Timestamp', 'Imported_energy (kWh)', 'Flexible_consumption (kWh)', 'Flexibility_demand_amount [-1;1]', 'Flexibility_demand_price (CZK/MWh)']
	Battery's headers: ['DeviceID', 'Timestamp', 'Stored_energy (kWh)']
	Battery parameters's headers: ['DeviceID', 'Battery_capacity (kWh)', 'Max_charge_rate (kW)', ' Init_capacity (%)']
	Weather's headers: ['Timestamp', 't_2m:C', 't_apparent:C', 'relative_humidity_2m:p', 'precip_15min:mm', 'diffuse_rad:W', 'direct_rad:W', 'global_rad:W', 'wind_speed_10m:ms', 'wind_dir_10m:d', 'uv:idx']
	Spot Market Prices's headers: ['Timestamp',

In [15]:
print("Merged DataFrame:")
data = merge_data(file_path)
data

Merged DataFrame:


,DeviceID,Timestamp,Imported_energy (kWh)_x,Exported_energy (kWh),Predicted_Imported_energy (kWh),Predicted_Exported_energy (kWh),Imported_energy (kWh)_y,Flexible_consumption (kWh),Flexibility_demand_amount [-1;1],Flexibility_demand_price (CZK/MWh),...,relative_humidity_2m:p,precip_15min:mm,diffuse_rad:W,direct_rad:W,global_rad:W,wind_speed_10m:ms,wind_dir_10m:d,uv:idx,Spot_Price (EUR/MWh),Spot_Price (CZK/MWh)
0,OM1,2024-01-01 00:00:00,0.044,0.0,0.038339,0.0,0.044,0.000,-0.456757,921.550783,...,99.3,0.00,0.0,0.0,0.0,0.7,9.8,0.0,"0,1","2,47"
1,OM1,2024-01-01 00:05:00,0.040,0.0,0.036386,0.0,0.040,0.000,-0.617518,855.522083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OM1,2024-01-01 00:10:00,0.040,0.0,0.037537,0.0,0.040,0.000,-0.915563,1007.028267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OM1,2024-01-01 00:15:00,0.038,0.0,0.045759,0.0,0.038,0.000,-0.506675,883.924399,...,97.6,0.03,0.0,0.0,0.0,1.1,48.9,0.0,NaN,NaN
4,OM1,2024-01-01 00:20:00,0.056,0.0,0.057124,0.0,0.056,0.000,-0.460941,576.059459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44635,OM5,2024-01-31 23:35:00,0.083,0.0,0.095046,0.0,0.083,0.043,-0.025611,-949.679349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44636,OM5,2024-01-31 23:40:00,0.082,0.0,0.078978,0.0,0.082,0.042,-0.030342,-1328.257315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44637,OM5,2024-01-31 23:45:00,0.073,0.0,0.081320,0.0,0.073,0.033,-0.028628,-1424.861971,...,95.5,0.00,0.0,0.0,0.0,1.6,202.0,0.0,NaN,NaN
44638,OM5,2024-01-31 23:50:00,0.074,0.0,0.083856,0.0,0.074,0.034,-0.028351,-829.226439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
